<a href="https://colab.research.google.com/github/James-R-Chapman/Kia-Invoices/blob/main/Kia_Invoices.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install camelot-py[cv]
#!apt-get install ghostscript
#!sudo apt install tesseract-ocr
!pip install openpyxl
!pip install PyPDF2
!pip install PyPDF4

In [ ]:
import re
import PyPDF4

pdf_upload = '/content/drive/MyDrive/Courage Kia/Kia Invoices'
# Open the PDF file
for filename in os.listdir(pdf_upload):
    if filename.endswith(".pdf"):
        pdf_path = os.path.join(pdf_upload, filename)
        with open(pdf_path, 'rb') as f:
            # Create a PDF reader object
            pdf_reader = PyPDF4.PdfFileReader(f)

            # Get the number of pages in the PDF
            num_pages = pdf_reader.getNumPages()
            print(f"Processing {filename}, {num_pages} pages")

            # Loop through each page in the PDF and search for the text "PAGE: 1"
            for i in range(num_pages):
                page = pdf_reader.getPage(i)
                text = page.extractText()
                if re.search(r'PAGE:\s*1', text):
                    print(f'"PAGE: 1" found on page {i+1}')



In [85]:
#!!! This Works!!!
import os
import re
import PyPDF4
from io import BytesIO
import csv

pdf_upload = '/content/drive/MyDrive/Courage Kia/Kia Invoices'
output_folder = '/content/drive/MyDrive/Courage Kia/Kia Invoices/Processed Kia Invoices'

if not os.path.exists(output_folder):
    os.makedirs(output_folder)

# Keep track of processed pages for each invoice
processed_invoices = set()

# Loop through each PDF file in the input folder
for filename in os.listdir(pdf_upload):
    x = 1
    if filename.endswith(".pdf"):
        # Define the full path of the PDF file
        pdf_path = os.path.join(pdf_upload, filename)

        # Extract the invoice number and date from the PDF file
        with open(pdf_path, 'rb') as f:
            # Create a PDF reader object
            pdf_reader = PyPDF4.PdfFileReader(f)

            # Get the number of pages in the PDF
            num_pages = pdf_reader.getNumPages()
            print(f"Processing {filename}, {num_pages} pages")

            # Loop through each page in the PDF and extract the invoice number and date
            for i in range(num_pages):
                page = pdf_reader.getPage(i)
                text = page.extractText()

                # Use regex to locate the invoice number and date on the page
                match_invoice = re.search(r'[C,P][A-Z]\d{8}', text)
                match_date = re.search(r'\d{1,2}/\d{1,2}/\d{2}', text)
                match_page = re.search(r'PAGE:\s*1', text)

                # If both the invoice number and date are found on the page
                if match_invoice and match_date:
                    # Extract the invoice number and date from the regex match objects
                    invoice_number = match_invoice.group()
                    date = match_date.group().replace("/", "-")
                    invoice_id = f"{invoice_number}_{date}"

                    # Define the output filename
                    invoice_filename = os.path.join(output_folder, f'{invoice_id}.pdf')
                    
                    # Create a new PDF writer object and add the current page to it
                    pdf_writer = PyPDF4.PdfFileWriter()
                    pdf_writer.addPage(page)

                    # Write the page for the invoice to a BytesIO object
                    buffer = BytesIO()
                    with open(pdf_path, 'rb') as f:
                        buffer.write(f.read())
                    pdf_reader = PyPDF4.PdfFileReader(buffer)

                    # Create a new PDF writer object and add the page from the buffer to it
                    pdf_writer = PyPDF4.PdfFileWriter()
                    pdf_writer.addPage(pdf_reader.getPage(i))

                    # If the output file already exists, add the current page to it
                    if os.path.exists(invoice_filename):
                        with open(invoice_filename, 'rb') as f_in:
                            try:
                                existing_pdf = PyPDF4.PdfFileReader(f_in)
                                num_existing_pages = existing_pdf.getNumPages()
                                
                                # Create a new PDF writer object and add the existing pages to it
                                pdf_writer = PyPDF4.PdfFileWriter()
                                for page_num in range(num_existing_pages):
                                    pdf_writer.addPage(existing_pdf.getPage(page_num))

                                # Add the current page to the PDF writer object
                                pdf_writer.addPage(page)

                                # Write the updated PDF file to a BytesIO object
                                buffer = BytesIO()
                                pdf_writer.write(buffer)

                                # Write the updated PDF file to disk
                                with open(invoice_filename, 'wb') as f_out:
                                    f_out.write(buffer.getvalue())
                                    x = x + 1
                                    print(f"Added {filename} page {i+1} to {invoice_id} page {x}")
                            except Exception as e:
                                print(f"Failed to add existing pages to {invoice_filename}: {e}")
                    else:
                        # Create a new PDF writer object and add the current page to it
                        pdf_writer = PyPDF4.PdfFileWriter()
                        pdf_writer.addPage(page)

                        # Write the new PDF file to disk
                        with open(invoice_filename, 'wb') as f_out:
                            pdf_writer.write(f_out)
                            x = 1
                            print(f"Used {filename} page {i+1} to {invoice_id} page {x}")

                    # Add the invoice to the set of processed invoices
                    processed_invoices.add(invoice_id)


Processing Batch Report (40).pdf, 17 pages
Used Batch Report (40).pdf page 1 to PL34389333_4-04-23 page 1
Used Batch Report (40).pdf page 2 to PL34388952_4-04-23 page 1
Used Batch Report (40).pdf page 3 to PL34388462_4-04-23 page 1
Used Batch Report (40).pdf page 4 to PL34388461_4-04-23 page 1
Used Batch Report (40).pdf page 5 to PL34388460_4-04-23 page 1
Used Batch Report (40).pdf page 6 to PL34388459_4-04-23 page 1
Added Batch Report (40).pdf page 7 to PL34388459_4-04-23 page 2
Used Batch Report (40).pdf page 8 to PL34387086_4-04-23 page 1
Used Batch Report (40).pdf page 9 to PL34385716_4-04-23 page 1
Used Batch Report (40).pdf page 10 to PL34385121_4-03-23 page 1
Used Batch Report (40).pdf page 11 to PL34385120_4-03-23 page 1
Used Batch Report (40).pdf page 12 to PL34385119_4-03-23 page 1
Used Batch Report (40).pdf page 13 to PL34383797_4-03-23 page 1
Used Batch Report (40).pdf page 14 to PL34382814_4-03-23 page 1
Used Batch Report (40).pdf page 15 to PL34382294_4-03-23 page 1
Added

In [ ]:
#!!! This Works!!!

import os
import re
import PyPDF4
from io import BytesIO

pdf_upload = '/content/drive/MyDrive/Courage Kia/Kia Invoices'
output_folder = '/content/drive/MyDrive/Courage Kia/Kia Invoices/Processed Kia Invoices'

if not os.path.exists(output_folder):
    os.makedirs(output_folder)

# Keep track of processed pages for each invoice
processed_invoices = set()

# Loop through each PDF file in the input folder
for filename in os.listdir(pdf_upload):
    x = 1
    if filename.endswith(".pdf"):
        # Define the full path of the PDF file
        pdf_path = os.path.join(pdf_upload, filename)

        # Extract the invoice number and date from the PDF file
        with open(pdf_path, 'rb') as f:
            # Create a PDF reader object
            pdf_reader = PyPDF4.PdfFileReader(f)

            # Get the number of pages in the PDF
            num_pages = pdf_reader.getNumPages()
            print(f"Processing {filename}, {num_pages} pages")

            # Loop through each page in the PDF and extract the invoice number and date
            for i in range(num_pages):
                page = pdf_reader.getPage(i)
                text = page.extractText()

                # Use regex to locate the invoice number and date on the page
                match_invoice = re.search(r'[C,P][A-Z]\d{8}', text)
                match_date = re.search(r'\d{1,2}/\d{1,2}/\d{2}', text)
                match_page = re.search(r'PAGE:\s*1', text)

                # If both the invoice number and date are found on the page
                if match_invoice and match_date:
                    # Extract the invoice number and date from the regex match objects
                    invoice_number = match_invoice.group()
                    date = match_date.group().replace("/", "-")
                    invoice_id = f"{invoice_number}_{date}"

                    # Define the output filename
                    invoice_filename = os.path.join(output_folder, f'{invoice_id}.pdf')
                    
                    # Create a new PDF writer object and add the current page to it
                    pdf_writer = PyPDF4.PdfFileWriter()
                    pdf_writer.addPage(page)

                    # Write the page for the invoice to a BytesIO object
                    buffer = BytesIO()
                    with open(pdf_path, 'rb') as f:
                        buffer.write(f.read())
                    pdf_reader = PyPDF4.PdfFileReader(buffer)

                    # Create a new PDF writer object and add the page from the buffer to it
                    pdf_writer = PyPDF4.PdfFileWriter()
                    pdf_writer.addPage(pdf_reader.getPage(i))

                    # If the output file already exists, add the current page to it
                    if os.path.exists(invoice_filename):
                        with open(invoice_filename, 'rb') as f_in:
                            try:
                                existing_pdf = PyPDF4.PdfFileReader(f_in)
                                for page_num in range(existing_pdf.getNumPages()):
                                    pdf_writer.insertPage(existing_pdf.getPage(page_num), 0)
                            except Exception as e:
                                print(f"Failed to add existing pages to {invoice_filename}: {e}")
                            
                            # Append the current page to the existing PDF file
                            with open(invoice_filename, 'ab') as f_out:
                                pdf_writer.write(f_out)
                                x = x + 1
                                print(f"Appended page {x} to {invoice_id}")
                    else:
                        # Create a new PDF writer object and add the current page to it
                        with open(invoice_filename, 'wb') as f_out:
                            pdf_writer.write(f_out)
                            x = 1
                            print(f"Created {invoice_id}")

                       
                        processed_invoices.add(invoice_id)

                        

In [79]:
!!!!! WORKING BACKUP!!!!

import os
import glob
import re
import PyPDF4
import csv
import camelot
import pandas as pd
from pdfminer.high_level import extract_text
from google.colab import files
import openpyxl
from google.colab import drive
from io import BytesIO

drive.mount('/content/drive')

pdf_upload = '/content/drive/MyDrive/Courage Kia/Kia Invoices'
output_folder = '/content/drive/MyDrive/Courage Kia/Kia Invoices/Processed Kia Invoices'

# Create output folder if it doesn't exist
if not os.path.exists(output_folder):
    os.makedirs(output_folder)

# Loop through each PDF file in the input folder
for filename in os.listdir(pdf_upload):
    x = 0
    current_invoice_id = None
    current_output_pdf = None
    if filename.endswith(".pdf"):
        # Define the full path of the PDF file
        pdf_path = os.path.join(pdf_upload, filename)

        # Open the PDF file in read-binary mode
        with open(pdf_path, 'rb') as f:
            # Create a PDF reader object
            pdf_reader = PyPDF4.PdfFileReader(f)

            # Get the number of pages in the PDF
            num_pages = pdf_reader.getNumPages()
            print(f"Processing {filename}, {num_pages} pages")

            # Loop through each page in the PDF
            for i in range(num_pages):
                # Get the current page and extract the text
                page = pdf_reader.getPage(i)
                text = page.extractText()

                # Use regex to locate the invoice number and date on the page
                match_invoice = re.search(r'[C,P][A-Z]\d{8}', text)
                match_date = re.search(r'\d{1,2}/\d{1,2}/\d{2}', text)

                # If both the invoice number and date are found on the page
                if match_invoice and match_date:
                    # Extract the invoice number and date from the regex match objects
                    invoice_number = match_invoice.group()
                    date = match_date.group().replace("/", "-")
                    invoice_id = f"{invoice_number}_{date}"

                    # If current_invoice_id is None or does not match invoice_id, close the current output PDF and create a new one
                    if current_invoice_id is None or current_invoice_id != invoice_id:
                        if current_output_pdf is not None:
                            current_output_pdf.close()
                        invoice_filename = os.path.join(output_folder, f'{invoice_id}.pdf')
                        current_output_pdf = open(invoice_filename, 'wb')
                        current_invoice_id = invoice_id
                        x = 1
                        print(f"Used {filename} page {i+1} to {invoice_id} page {x}")
                    
                    # Create a new PDF writer object and add the current page to it
                    pdf_writer = PyPDF4.PdfFileWriter()
                    pdf_writer.addPage(page)

                    # Write the page for the invoice to a BytesIO object
                    buffer = BytesIO()
                    with open(pdf_path, 'rb') as f:
                        buffer.write(f.read())
                    pdf_reader = PyPDF4.PdfFileReader(buffer)

                    # Create a new PDF writer object and add the page from the buffer to it
                    pdf_writer = PyPDF4.PdfFileWriter()
                    pdf_writer.addPage(pdf_reader.getPage(i))

                    # Write the contents of the BytesIO object to a new PDF file
                    invoice_filename = os.path.join(output_folder, f'{invoice_id}.pdf')
                    if os.path.exists(invoice_filename):
                        with open(invoice_filename, 'rb') as f_in:
                            try:
                                existing_pdf = PyPDF4.PdfFileReader(f_in)
                                for page_num in range(existing_pdf.getNumPages()):
                                    pdf_writer.insertPage(existing_pdf.getPage(page_num), 0)
                            except Exception as e:
                                #print(f"Failed to add existing pages to {invoice_id}: {e}")
                                continue  # continue the loop execution without appending the current page
                                                
                            # Append the current page to the existing PDF file
                            with open(invoice_filename, 'ab') as f_out:
                                pdf_writer.write(f_out)
                                x = x + 1
                                print(f"Added {filename} page {i+1} to {invoice_id} page {x}")
                    else:
                        # If the file doesn't exist, create it and write the current page to it
                        with open(invoice_filename, 'wb') as f_out:
                            pdf_writer.write(f_out)
                            x = 0
                            print(f"Used {filename} page {i+1} to {invoice_id}")



Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
Processing Batch Report (40).pdf, 17 pages
Used Batch Report (40).pdf page 1 to PL34389333_4-04-23 page 1
Used Batch Report (40).pdf page 2 to PL34388952_4-04-23 page 1
Used Batch Report (40).pdf page 3 to PL34388462_4-04-23 page 1
Used Batch Report (40).pdf page 4 to PL34388461_4-04-23 page 1
Used Batch Report (40).pdf page 5 to PL34388460_4-04-23 page 1
Used Batch Report (40).pdf page 6 to PL34388459_4-04-23 page 1
Used Batch Report (40).pdf page 8 to PL34387086_4-04-23 page 1
Used Batch Report (40).pdf page 9 to PL34385716_4-04-23 page 1
Used Batch Report (40).pdf page 10 to PL34385121_4-03-23 page 1
Used Batch Report (40).pdf page 11 to PL34385120_4-03-23 page 1
Used Batch Report (40).pdf page 12 to PL34385119_4-03-23 page 1
Used Batch Report (40).pdf page 13 to PL34383797_4-03-23 page 1
Used Batch Report (40).pdf page 14 to PL34382814_4-03-23 page 1
Used

In [ ]:
import os
import glob
import csv
from pdfminer.high_level import extract_text
#import time
import pickle

pdf_folder = '/content/drive/MyDrive/Courage Kia/Kia Invoices'
csv_file_path = '/content/drive/MyDrive/Courage Kia/Invoice Rec/parts_invoices.csv'
backup_csv_file_path = '/content/drive/MyDrive/Courage Kia/Invoice Rec/processed_invoices_backup.csv'

headers = ['Invoice Number', 'Invoice Date', 'Line #', 'Order #', 'Part Number', 'Status', 'QTY Order', 'QTY Ship', 'B/O Cancel', 'Cost', 'Ext Price']
invoice_headers = ['INVOICE #', 'DATE', 'SHIPMENT', 'CORE', 'SUBTOTAL', 'FREIGHT', 'HANDLING', 'MISC', 'RESTOCK FEE', 'TAX', 'INVOICE TOTAL']
credit_headers = ['Invoice Number', 'Invoice Date', 'Line #', 'Order #', 'Part Number', 'Type', 'QTY', 'ACT Code', 'Handling Credit', 'Cost', 'Ext Price']
kia_invoices = []
invoices = []
credits = []
prev_row = []
invoice_pages = set()  # create a set to store all unique invoice numbers encountered
invoice_no = ''
date = ''
line = ''
ro = ''
order = ''
part = ''
number = ''
part_number = ''
status = ''
qty_order = ''
act_code = ''
qty_ship = ''
handling = ''
bo_cancel = ''
cost = ''
price = ''
page = ''
core = ''
part_number = part + number
shipment = ''
invoice_core = ''
freight = ''
invoice_handling = ''
misc = ''
restock = ''
tax = ''
invoice_total = ''
prev_invoice_no = ''

# Load the existing database, if it exists
if os.path.exists('processed_files.pkl'):
    with open('processed_files.pkl', 'rb') as f:
        processed_files = pickle.load(f)
else:
    processed_files = {}

while True:
    # Get a list of all PDF files in the folder
    pdf_files = glob.glob(pdf_folder + '/*.pdf')

    # Filter out files that have already been processed
    new_files = [file for file in pdf_files if os.path.basename(file) not in processed_files]

    # Process each new PDF file
    for pdf_path in new_files:
        # Process the PDF file as before
        print(f"Reading file: {pdf_path}")
        
        # Extract the text from the PDF file using pdfminer
        text = extract_text(pdf_path)

        # Split the text into lines and remove any empty lines
        lines = [line.strip() for line in text.split('\n') if line.strip()]

        # Write the lines to the CSV file in append mode
        with open(csv_file_path, 'a', newline='') as csv_file, \
                open(backup_csv_file_path, 'a', newline='') as backup_csv_file:
            writer = csv.writer(csv_file)
            backup_writer = csv.writer(backup_csv_file)
            for page in pages:
                skip_page = False
                for row in [line.split() for line in page if line.strip()]:
                    if len(row) >= 6 and (row[0].startswith('Courage') and 'CLAIM' in row[3]) or (row[0].startswith('4290') and row[2].startswith('BLVD') and row[3].startswith('I') and row[4].startswith('N')):
                        invoice_no = row[5]
                    elif len(row) >= 4 and row[0].startswith('DATE:') and 'PAGE:' in row[2]:
                        date = row[1]
                        page = int(row[3])
                    elif len(row) >= 5 and row[0].startswith('GASTONIA') and ('DATE' in row[3] or 'Date' in row[3]):
                        date = row[4]
                    elif row[0].startswith('SUB') or row[0].startswith('Sub'):
                        subtotal = row[2]
                    elif len(row) >= 5 and (row[3].startswith('PART') or row[3].startswith('Part')) and (row[4].startswith('TOTAL') or row[4].startswith('Total')):
                        subtotal = row[5]
                    elif len(row) >= 5 and (row[2].startswith('PART') or row[2].startswith('Part')) and (row[3].startswith('TOTAL') or row[3].startswith('Total')):
                        subtotal = row[4]
                    elif len(row) >= 4 and (row[1].startswith('PART') or row[1].startswith('Part')) and (row[2].startswith('TOTAL') or row[2].startswith('Total')):
                        subtotal = row[3]
                    elif row[0].startswith('INVOICE') or row[0].startswith('Invoice') or row[0].startswith('CLAIM') or row[0].startswith('Claim'):
                        if len(row) >= 3:
                            invoice_total = row[2]
                                        # Check if the invoice has already been processed
                    if (invoice_no, date, page, invoice_total) in processed_files:
                        print(f"Skipping row with invoice number {invoice_no}, already exists in backup CSV file")
                        skip_page = True
                        break

                    # Add the row to the CSV file
                    writer.writerow(row)

                # If any row on the page has already been added to the CSV file, skip the entire page
                if skip_page:
                    continue

                # Add the invoice to the backup CSV file
                backup_writer.writerow([invoice_no, date, page, invoice_total])
                processed_files.add((invoice_no, date, page, invoice_total))

    # Wait for some time before scanning for new files again
    #time.sleep(60)  # wait for 1 minute

KeyboardInterrupt: ignored

In [ ]:
import glob
import csv
import os.path
from pdfminer.high_level import extract_text

pdf_folder = '/content/drive/MyDrive/Courage Kia/Kia Invoices'
csv_file_path = '/content/drive/MyDrive/Courage Kia/Invoice Rec/parts_invoices.csv'
backup_csv_file_path = '/content/drive/MyDrive/Courage Kia/Invoice Rec/processed_invoices_backup.csv'

# Define a function to read the data from the backup CSV file
# Define a function to read the data from the backup CSV file
def read_backup_csv(backup_csv_file_path):
    try:
        with open(backup_csv_file_path, 'r') as backup_csv_file:
            reader = csv.reader(backup_csv_file)
            return {(row[0], row[1], int(row[2]) if row[2].isdigit() else row[2], row[3]) for row in reader}
    except FileNotFoundError:
        with open(backup_csv_file_path, 'w', newline='') as backup_csv_file:
            writer = csv.writer(backup_csv_file)
            writer.writerow(['invoice_no', 'date', 'page', 'invoice_total'])
        return set()


# Read the existing .csv file to add the relevant information for already processed invoices to a set
processed_invoices = read_backup_csv(backup_csv_file_path)

# Process each PDF file in the folder
for pdf_path in glob.glob(pdf_folder + '/*.pdf'):
    print(f"Processing file: {pdf_path}")
    
    # Extract the text from the PDF file using pdfminer
    text = extract_text(pdf_path)

    # Split the text into pages, each containing a list of lines
    pages = [line.strip() for line in text.split('\n') if line.strip()]

    # Check if the invoice has already been processed
    skip_pdf = False
    for page in pages:
        invoice_no, date, page_num, invoice_total = '', '', '', ''
        for row in [line.split() for line in page if line.strip()]:
            if len(row) >= 6 and (row[0].startswith('Courage') and 'CLAIM' in row[3]) or (row[0].startswith('4290') and row[2].startswith('BLVD') and row[3].startswith('I') and row[4].startswith('N')):
                invoice_no = row[5]
            elif len(row) >= 4 and row[0].startswith('DATE:') and 'PAGE:' in row[2]:
                date = row[1]
                page_num = int(row[3])
            elif len(row) >= 5 and row[0].startswith('GASTONIA') and ('DATE' in row[3] or 'Date' in row[3]):
                date = row[4]
            elif row[0].startswith('INVOICE') or row[0].startswith('Invoice') or row[0].startswith('CLAIM') or row[0].startswith('Claim'):
                if len(row) >= 3:
                    invoice_total = row[2]

        if (invoice_no, date, page_num, invoice_total) in processed_invoices:
            print(f"Skipping PDF with invoice number {invoice_no}, already exists in backup CSV file")
            skip_pdf = True
            break

    if skip_pdf:
        continue

    # Write the rows to the CSV file in append mode
    with open(csv_file_path, 'a', newline='') as csv_file, open(backup_csv_file_path, 'a', newline='') as backup_csv_file:
        writer = csv.writer(csv_file)
        backup_writer = csv.writer(backup_csv_file)
        # Process each PDF file in the folder
    for pdf_path in glob.glob(pdf_folder + '/*.pdf'):
        print(f"Processing file: {pdf_path}")
        
        # Extract the text from the PDF file using pdfminer
        text = extract_text(pdf_path)

        # Split the text into pages, each containing a list of lines
        pages = [line.strip() for line in text.split('\n') if line.strip()]

        # Check if the invoice has already been processed by reading the backup CSV file
        skip_file = False
        with open(backup_csv_file_path, 'r') as backup_csv_file:
            reader = csv.reader(backup_csv_file)
            for row in reader:
                if row[0] and row[1] and row[2] and row[3]:
                    if row[0] in text and row[1] in text and row[2] in text and row[3] in text:
                        skip_file = True
                        print(f"Invoice {row[0]} already processed in a previous PDF file. Skipping file {pdf_path}")
                        break

        if skip_file:
            continue

        # Write the rows to the CSV file in append mode, skipping already processed invoices
        for page in pages:
            invoicepdf = []
            skip_page = False
            for row in [line.split() for line in page if line.strip()]:
                if len(row) >= 6 and (row[0].startswith('Courage') and 'CLAIM' in row[3]) or (row[0].startswith('4290') and row[2].startswith('BLVD') and row[3].startswith('I') and row[4].startswith('N')):
                    invoice_no = row[5]
                elif len(row) >= 4 and row[0].startswith('DATE:') and 'PAGE:' in row[2]:
                    date = row[1]
                    page = int(row[3])
                elif len(row) >= 5 and row[0].startswith('GASTONIA') and ('DATE' in row[3] or 'Date' in row[3]):
                    date = row[4]
                elif row[0].startswith('SUB') or row[0].startswith('Sub'):
                    subtotal = row[2]
                elif len(row) >= 5 and (row[3].startswith('PART') or row[3].startswith('Part')) and (row[4].startswith('TOTAL') or row[4].startswith('Total')):
                    subtotal = row[5]
                elif len(row) >= 5 and (row[2].startswith('PART') or row[2].startswith('Part')) and (row[3].startswith('TOTAL') or row[3].startswith('Total')):
                    subtotal = row[4]
                elif len(row) >= 4 and (row[1].startswith('PART') or row[1].startswith('Part')) and (row[2].startswith('TOTAL') or row[2].startswith('Total')):
                    subtotal = row[3]
                elif row[0].startswith('INVOICE') or row[0].startswith('Invoice') or row[0].startswith('CLAIM') or row[0].startswith('Claim'):
                    if len(row) >= 3:
                        invoice_total = row[2]

                # Check if the invoice has already been processed
                if (invoice_no, date, page, invoice_total) in processed_invoices:
                    print(f"Skipping invoice with number {invoice_no}, already exists in backup CSV file")
                    skip_invoice = True
                    break
        # Check if the invoice has already been processed
        skip_invoice = (invoice_no, date, page, invoice_total) in processed_invoices

        # Skip the current PDF if its invoice has already been processed
        if skip_invoice:
            continue

        # Write the rows to the CSV file in append mode
        for page in pages:
            writer.writerows([line.split() for line in page.split('\n') if line.strip()])

        # Write the invoice to the backup CSV file and add it to the processed invoices set
        invoice_info = [invoice_no, date, page, invoice_total]
        backup_writer.writerow(invoice_info)
        processed_invoices.add(tuple(invoice_info))

print("Invoice processing complete!")

Processing file: /content/drive/MyDrive/Courage Kia/Kia Invoices/Batch Report (40).pdf
Processing file: /content/drive/MyDrive/Courage Kia/Kia Invoices/Batch Report (40).pdf


ValueError: ignored

In [ ]:
import os
import glob
import csv
from typing import Tuple

pdf_folder = '/content/drive/MyDrive/Courage Kia/Kia Invoices'
csv_file_path = '/content/drive/MyDrive/Courage Kia/Invoice Rec/parts_invoices.csv'
backup_csv_file_path = '/content/drive/MyDrive/Courage Kia/Invoice Rec/backup.csv'

# Create a set to store all unique invoice numbers encountered
processed_invoices = set()

for pdf_path in glob.glob(pdf_folder + '/*.pdf'):
    if not skip:
        print(f"Processing file: {pdf_path}")

        # Extract the text from the PDF file using pdfminer
        text = extract_text(pdf_path)

        # Split the text into pages
        pages = text.split('\f')

        # Extract the invoice data from each page
        for page in pages:
            invoicepdf = extract_invoice_data(page)

            # Skip the page if it does not contain an invoice
            if not invoicepdf:
                continue

            # Check if the invoice has already been processed
            if (invoice_no, date, page, invoice_total) in processed_invoices:
                print(f"Skipping duplicate invoice: {invoice_no}, {date}, {page}, {invoice_total}")
                continue

            # Add the invoice to the processed invoices set
            processed_invoices.add((invoice_no, date, page, invoice_total))

            # Check if the invoice has already been added to parts_invoice.csv
            with open(backup_csv_file_path, 'r', newline='') as backup_csv_file:
                backup_reader = csv.reader(backup_csv_file)
                for row in backup_reader:
                    if (invoice_no, date, row[2], row[-1]) == (invoicepdf['Invoice Number'], invoicepdf['Invoice Date'], row[3], row[-2]):
                        print(f"Invoice already in backup: {invoicepdf['Invoice Number']}, {invoicepdf['Invoice Date']}, {row[2]}, {row[-1]}")
                        skip_invoice = True
                        break
                else:
                    skip_invoice = False

            # Skip the current PDF if its invoice has already been processed
            if skip_invoice:
                continue

            # Write the rows to the CSV file in append mode
            with open(csv_file_path, 'a', newline='') as csv_file:
                writer = csv.writer(csv_file)
                for line in invoicepdf['lines']:
                    writer.writerow(line)

            # Write the invoice to the backup CSV file and add it to the processed invoices set
            with open(backup_csv_file_path, 'a', newline='') as backup_csv_file:
                backup_writer = csv.writer(backup_csv_file)
                backup_writer.writerow(invoicepdf.values())
                
print("Done processing invoices.")


Done processing invoices.


# ***DONT DELETE BELOW ME***

In [ ]:
import csv
import glob
import re
import camelot
import pandas as pd
from pdfminer.high_level import extract_text
from google.colab import files
import openpyxl
from google.colab import drive
import os
drive.mount('/content/drive')

# Upload the PDF file
#uploaded = files.upload()

# Get the path of the uploaded file
#pdf_path = list(uploaded.keys())[0]
#pdf_path = '/content/Batch Report (39).pdf'

pdf_folder = '/content/drive/MyDrive/Courage Kia/Kia Invoices'
csv_file_path = '/content/drive/MyDrive/Courage Kia/Invoice Rec/parts_invoices.csv'
# Check if parts_invoices.csv file already exists
if os.path.isfile(csv_file_path):
    skip = True
    print(f"{csv_file_path} already exists. Skip is set to {skip}.")
else:
    skip = False
    print(f"{csv_file_path} does not exist. Skip is set to {skip}.")

subtotal = 0
for pdf_path in glob.glob(pdf_folder + '/*.pdf'):

    if not skip:
        print(f"Reading file: {pdf_path}")
        
        # Extract the text from the PDF file using pdfminer
        text = extract_text(pdf_path)

        # Split the text into lines and remove any empty lines
        lines = [line.strip() for line in text.split('\n') if line.strip()]

        # Write the lines to the CSV file in append mode
        with open(csv_file_path, 'a', newline='') as csv_file:
            writer = csv.writer(csv_file)
            writer.writerows([line.split() for line in lines])

headers = ['Invoice Number', 'Invoice Date', 'Line #', 'Order #', 'Part Number', 'Status', 'QTY Order', 'QTY Ship', 'B/O Cancel', 'Cost', 'Ext Price']
invoice_headers = ['INVOICE #', 'DATE', 'SHIPMENT', 'CORE', 'SUBTOTAL', 'FREIGHT', 'HANDLING', 'MISC', 'RESTOCK FEE', 'TAX', 'INVOICE TOTAL']
credit_headers = ['Invoice Number', 'Invoice Date', 'Line #', 'Order #', 'Part Number', 'Type', 'QTY', 'ACT Code', 'Handling Credit', 'Cost', 'Ext Price']
kia_invoices = []
invoices = []
credits = []
prev_row = []
invoice_pages = set()  # create a set to store all unique invoice numbers encountered
invoice_no = ''
date = ''
line = ''
ro = ''
order = ''
part = ''
number = ''
part_number = ''
status = ''
qty_order = ''
act_code = ''
qty_ship = ''
handling = ''
bo_cancel = ''
cost = ''
price = ''
page = ''
core = ''
part_number = part + number
shipment = ''
invoice_core = ''
freight = ''
invoice_handling = ''
misc = ''
restock = ''
tax = ''
invoice_total = ''
prev_invoice_no = ''

for i, row in enumerate(csv.reader(open('/content/drive/MyDrive/Courage Kia/Invoice Rec/parts_invoices.csv', 'r'))):
    core = ''
    if len(row) >= 4 and (row[0].startswith('Courage') and 'CLAIM' in row[3]) or (row[0].startswith('4290') and row[2].startswith('BLVD') and row[3].startswith('I') and row[4].startswith('N')):
        invoice_no = row[5]
        invoice_core = ''
        freight = ''
        invoice_handling = ''
        misc = ''    
    elif len(row) >= 4 and row[0].startswith('DATE:') and 'PAGE:' in row[2]:
        date = row[1]
        page = row[3]
    elif len(row) >= 5 and row[0].startswith('GASTONIA') and ('DATE' in row[3] or 'Date' in row[3]):
        date = row[4]
    elif row[0].startswith('SHIPMENT'):
        if len(row) >= 6:
            shipment = row[2]
    elif row[0].startswith('***Core'):
        if len(row) >= 4:
            invoice_core = row[3]
    elif row[0].startswith('SUB') or row[0].startswith('Sub'):
            subtotal = row[2]
    elif len(row) >= 5 and (row[3].startswith('PART') or row[3].startswith('Part')) and (row[4].startswith('TOTAL') or row[4].startswith('Total')):
            subtotal = row[5]
    elif len(row) >= 5 and (row[2].startswith('PART') or row[2].startswith('Part')) and (row[3].startswith('TOTAL') or row[3].startswith('Total')):
            subtotal = row[4]
    elif len(row) >= 4 and (row[1].startswith('PART') or row[1].startswith('Part')) and (row[2].startswith('TOTAL') or row[2].startswith('Total')):
            subtotal = row[3]
    elif row[0].startswith('FREIGHT'):
        if len(row) >= 2:
            freight = row[1]
    elif row[0].startswith('HANDLING'):
        if len(row) >= 2:
            invoice_handling = row[1]
    elif row[0].startswith('MISC'):
        if len(row) >= 2:
            misc = row[1]
    elif row[0].startswith('RSTK'):
        if len(row) >=3:
            restock = row[2]
    elif row[0].startswith('CODES') or row[0].startswith('TAX') or row[0].startswith('Tax'):
        if len(row) >= 6:
            tax = row[5]
    elif row[0].startswith('INVOICE') or row[0].startswith('Invoice') or row[0].startswith('CLAIM') or row[0].startswith('Claim'):
        if len(row) >= 3:
            invoice_total = row[2]
    elif len(row) >= 5 and row[0].startswith('4290') and 'CLAIM' in row[4]:
        order = row[6]
    elif len(row) >= 5 and row[0].startswith('ORDER') and 'ORDER' in row[3]:
        order = row[2]
    elif len(row) >= 2 and (row[0].startswith('PAGE:') or row[0].startswith('Page:')) and row[1].isdigit():
        page = row[1]
    elif len(row) >= 7 and row[0].startswith('Trk#:') and row[4].isdigit():
        status = row[1]
        ro = row[4]
    elif len(row) >= 7 and row[0].startswith('Trk#:') and row[5].isdigit():
        status = row[1]
        ro = row[5]
    elif i >= 16 and len(row) == 5 and row[0].startswith('Core') and row[1].startswith('Charge'):
        line = ''
        core = part_number
        part = row[0]
        number = row[1]
        status = ''
        qty_order = ''
        qty_ship = row[2]
        bo_cancel = ''
        cost = row[3]
        try:
            price = row[4]
        except IndexError:
            print(f"line 94 IndexError on row {i}: {row}")
        part_number = part + " " + number
        invoice_page = (invoice_no, page, part_number, line, core)
        if invoice_page not in invoice_pages:
            if isinstance(qty_order, str):
                if len(qty_order) > 0:
                    qty_order = float(qty_order)
                else:
                    qty_order = None
            if isinstance(qty_ship, str):
                if len(qty_ship) > 0:
                    qty_ship = float(qty_ship)
                else:
                    qty_ship = None
            if isinstance(cost, str):
                if len(cost) > 7:
                    cost = float(cost.replace(',', ''))
                elif len(cost) > 1:
                    cost = float(cost)
                else:
                    cost = None                
            if isinstance(price, str):
                if len(price) > 7:
                    price = float(price.replace(',', ''))
                elif len(price) > 1:
                    price = float(price)
                else:
                    price = None
            invoice_data = [invoice_no, date, line, order, part_number, status, qty_order, qty_ship, bo_cancel, cost, price]
            invoices.append(invoice_data)
            invoice_pages.add(invoice_page)  # add the invoice number to the set
        else:
            print(f"line 102 Duplicate invoice part row {i}: {invoice_page}")
    
    elif i >= 16 and len(row) == 4 and '(' in row[0] and ')' in row[2]:
        line = ''
        core = ''
        part = row[0]
        number = row[1] + ')'
        status = ''
        qty_order = row[3]
        qty_ship = ''
        bo_cancel = ''
        cost = ''
        price = ''
        part_number = part + number
        invoice_page = (invoice_no, page, part_number, line, core)
        if invoice_page not in invoice_pages:
            if isinstance(qty_order, str):
                if len(qty_order) > 0:
                    qty_order = float(qty_order)
                else:
                    qty_order = None
            if isinstance(qty_ship, str):
                if len(qty_ship) > 0:
                    qty_ship = float(qty_ship)
                else:
                    qty_ship = None
            if isinstance(cost, str):
                if len(cost) > 7:
                    cost = float(cost.replace(',', ''))
                elif len(cost) > 1:
                    cost = float(cost)
                else:
                    cost = None                
            if isinstance(price, str):
                if len(price) > 7:
                    price = float(price.replace(',', ''))
                elif len(price) > 1:
                    price = float(price)
                else:
                    price = None
            invoice_data = [invoice_no, date, line, order, part_number, status, qty_order, qty_ship, bo_cancel, cost, price]
            invoices.append(invoice_data)
            invoice_pages.add(invoice_page)  # add the invoice number to the set
            core = ''
        else:
            print(f"line 120 Duplicate invoice part row {i}: {invoice_page}")
    if len(row) >= 5 and not row[0].isdigit() and row[3].endswith(',000.00'):
        if invoice_no.startswith('C'):
            if row[4].endswith('CR'):# and row[2] < 9:
                line = ro
                order = status
                part = row[0]
                number = row[1]
                status = 'Core Return'
                qty_order = row[2]
                act_code = 'CR'
                handling = ''
                cost = row[3]
                core = part_number
                shipment = ''
                try:
                    price = row[4]
                    if price.endswith('CR'):
                        price = float(price.replace(',', '')[:-2]) * -1
                    else:
                        price = float(price.replace(',', ''))
                except IndexError:
                    print(f"line 134 IndexError on row {i}: {row}")
            part_number = part + number
            invoice_page = (invoice_no, page, part_number, line, core)
            if invoice_page not in invoice_pages:
                if isinstance(qty_order, str):
                    if len(qty_order) > 0:
                        qty_order = float(qty_order)
                    else:
                        qty_order = None
                if isinstance(handling, str):
                    if len(handling) > 0:
                        handling = (float(handling) * -1)
                    else:
                        handling = None
                if isinstance(cost, str):
                    if len(cost) > 7:
                        cost = (float(cost.replace(',', '')) * -1)
                    elif len(cost) > 1:
                        cost = (float(cost) * -1)
                    else:
                        cost = None                
                if isinstance(price, str):
                    if len(price) > 7:
                        price = (float(price.replace(',', '')) * -1)
                    elif len(price) > 1:
                        price = (float(price) * -1)
                    else:
                        price = None
                part_number = part + number
                credit_data = [invoice_no, date, line, order, part_number, status, qty_order, act_code, handling, cost, price]
                credits.append(credit_data)
                invoice_pages.add(invoice_page)  # add the invoice number to the set
    if i >= 16 and row[0].isdigit() and row[0] != '4290' and row[0] != '111':
        if len(row) <= 5:
            if invoice_no.startswith('C'):
                if row[4].endswith('CR'):
                    line = ro
                    order = status
                    part = row[0]
                    number = row[1]
                    status = 'Core Return'
                    qty_order = row[2]
                    act_code = 'CR'
                    handling = ''
                    cost = row[3]
                    core = part_number
                    shipment = ''
                    try:
                        price = row[4]
                        if price.endswith('CR'):
                            price = float(price.replace(',', '')[:-2]) * -1
                        else:
                            price = float(price.replace(',', ''))
                    except IndexError:
                        print(f"line 134 IndexError on row {i}: {row}")
        elif len(row) >= 6:
            line = row[0]
            part = row[1]
            number = row[2]
            status = row[3]
            if invoice_no.startswith('C'):
                if "CR" in row[4]:
                    line = ro
                    order = status
                    part = row[0]
                    number = row[1]
                    status = 'Core Return'
                    qty_order = row[2]
                    act_code = 'CR'
                    handling = ''
                    cost = row[3]
                    core = part_number
                    shipment = ''
                    try:
                        price = row[4]
                        if price.endswith('CR'):
                            price = float(price.replace(',', '')[:-2]) * -1
                        else:
                            price = float(price.replace(',', ''))
                    except IndexError:
                        print(f"line 155 IndexError on row {i}: {row}")
                else:
                    qty_order = row[4]
                    act_code = row[5]
                    handling = row[6]
                    cost = row[7]
                    shipment = ''
                    try:
                        price = row[8]
                    except IndexError:
                        print(f"line 164 IndexError on row {i}: {row}")
            elif invoice_no.startswith('P'):
                if not row[3].isdigit():
                    if row[3].startswith('A'):
                        qty_order = ''
                        qty_ship = row[4]
                        bo_cancel = ''
                        cost = row[5]
                        try:
                            price = row[6]
                        except IndexError:
                            print(f"line 175 IndexError on row {i}: {row}")
                    elif row[3].startswith('B'):
                        qty_order = row[4]
                        qty_ship = row[5]
                        bo_cancel = row[6]
                        cost = row[7]
                        try:
                            price = row[8]
                        except IndexError:
                            print(f"line 184 IndexError on row {i}: {row}")
                    elif row[3] == 'J':
                        qty_order = row[4]
                        qty_ship = row[5]
                        cost = row[6]
                        try:
                            price = row[7]
                        except IndexError:
                            print(f"line 192 IndexError on row {i}: {row}")
                    elif row[3] == 'H':
                        qty_order = row[4]
                        qty_ship = row[5]
                        cost = row[6]
                        try:
                            price = row[7]
                        except IndexError:
                            print(f"line 200 IndexError on row {i}: {row}")
                    elif row[3].startswith('Z'):
                        qty_order = row[4]
                        qty_ship = row[5]
                        bo_cancel = row[6]
                        cost = row[7]
                        try:
                            price = row[8]
                        except IndexError:
                            print(f"line 184 IndexError on row {i}: {row}")
                    elif row[3].endswith('-'):
                        row[3] = row[3][:-1]
                        row[4] = row[4][:-1]
                        row[6] = row[6][:-1]
                        status = ''
                        qty_order = (int(row[3]) * -1)
                        qty_ship = (int(row[4]) * -1)
                        bo_cancel = ''
                        cost = row[5]
                        price = '{:.2f}'.format(float(row[6].replace(',', '')) * -1)
                elif row[3].isdigit():
                    status = ''
                    qty_order = row[3]
                    qty_ship = row[4]
                    bo_cancel = ''
                    cost = row[5]
                    try:
                        price = row[6]
                    except IndexError:
                        print(f"line 210 IndexError on row {i}: {row}")
        part_number = part + number 
        if invoice_no.startswith('P'):
            invoice_page = (invoice_no, page, part_number, line, core)
            if isinstance(qty_order, str):
                if len(qty_order) > 0:
                    qty_order = float(qty_order)
                else:
                    qty_order = None
            if isinstance(qty_ship, str):
                if len(qty_ship) > 0:
                    qty_ship = float(qty_ship)
                else:
                    qty_ship = None
            if isinstance(cost, str):
                cost = float(cost.replace(',', ''))
                if isinstance(cost, float):
                    cost = float(cost)
                else:
                    cost = None                
            if isinstance(price, str):
                price = float(price.replace(',', ''))
                if isinstance(price, float):
                    price = float(price)
                else:
                    price = None
            invoice_data = [invoice_no, date, line, order, part_number, status, qty_order, qty_ship, bo_cancel, cost, price]
            invoices.append(invoice_data)
            invoice_pages.add(invoice_page)  # add the invoice number to the set
        elif invoice_no.startswith('C'):
            invoice_page = (invoice_no, page, part_number, line, core)
            if invoice_page not in invoice_pages:
                if isinstance(qty_order, str):
                    if len(qty_order) > 0:
                        qty_order = float(qty_order)
                    else:
                        qty_order = None
                if isinstance(handling, str):
                    if len(handling) > 0:
                        handling = (float(handling) * -1)
                    else:
                        handling = None
                if isinstance(cost, str):
                    if len(cost) > 7:
                        cost = (float(cost.replace(',', '')) * -1)
                    elif len(cost) > 1:
                        cost = (float(cost) * -1)
                    else:
                        cost = None                
                if isinstance(price, str):
                    if len(price) > 7:
                        price = (float(price.replace(',', '')) * -1)
                    elif len(price) > 1:
                        price = (float(price) * -1)
                    else:
                        price = None
                credit_data = [invoice_no, date, line, order, part_number, status, qty_order, act_code, handling, cost, price]
                credits.append(credit_data)
                invoice_pages.add(invoice_page)  # add the invoice number to the set
            #else:
            #    print(f"line 229 Duplicate invoice row {i}: {invoice_page}")
    if len(prev_row) > 0 and ('INVOICE' in prev_row[0] or 'Invoice' in prev_row[0] or 'CLAIM' in prev_row[0] or 'Claim' in prev_row[0]):
        if isinstance(invoice_core, str):
            if len(invoice_core) > 7:
                invoice_core = float(invoice_core.replace(',', ''))
            elif len(invoice_core) > 0:
                invoice_core = float(invoice_core)
            else:
                invoice_core = None
        if isinstance(subtotal, str):
            if subtotal.endswith('CR'):
                subtotal = float(subtotal.replace(',', '')[:-2]) * -1
            elif subtotal.endswith('-'):
                subtotal = float(subtotal.replace(',', '')[:-1]) * -1
        if isinstance(subtotal, str):
            if len(subtotal) > 7:
                subtotal = float(subtotal.replace(',', ''))
            elif len(subtotal) > 0:
                subtotal = float(subtotal)
            else:
                subtotal = None
        if isinstance(freight, str):
            if len(freight) > 0:
                freight = float(freight)
            else:
                freight = None
        if isinstance(invoice_handling, str):
            if len(invoice_handling) > 7:
                invoice_handling = float(invoice_handling.replace(',', ''))
            elif len(invoice_handling) > 0:
                invoice_handling = float(invoice_handling)
            else:
                invoice_handling = None                
        if isinstance(misc, str):
            if len(misc) > 7:
                misc = float(misc.replace(',', ''))
            elif len(misc) > 0:
                misc = float(misc)
            else:
                misc = None
        if isinstance(restock, str):
            if len(restock) > 7:
                restock = float(restock.replace(',', ''))
            elif len(restock) > 0:
                restock = float(restock)
            else:
                restock = None
        if isinstance(tax, str):
            if len(tax) > 7:
                tax = float(tax.replace(',', ''))
            elif len(tax) > 1:
                tax = float(tax)
            else:
                tax = None
        if isinstance(invoice_total, str):
            if invoice_total.endswith('CR'):
                invoice_total = float(invoice_total.replace(',', '')[:-2]) * -1
            elif invoice_total.endswith('-'):
                invoice_total = float(invoice_total.replace(',', '')[:-1]) * -1
        if isinstance(invoice_total, str):
            if len(invoice_total) > 7:
                invoice_total = float(invoice_total.replace(',', ''))
            elif len(invoice_total) > 1:
                invoice_total = float(invoice_total)
            else:
                invoice_total = None
        if invoice_no.startswith('C'):
            if subtotal > 0:
                subtotal = subtotal * -1
            if invoice_total > 0:
                invoice_total = invoice_total * -1
            if invoice_handling != None:
                invoice_handling = invoice_handling * -1
        kia_invoices.append([invoice_no, date, shipment, invoice_core, subtotal, freight, invoice_handling, misc, restock, tax, invoice_total])
    prev_row = row

# Create a new workbook
workbook = openpyxl.Workbook()

# Create the "Invoices" sheet
invoices_sheet = workbook.active
invoices_sheet.title = 'Invoices'
invoices_sheet.append(headers)
for invoice_data in invoices:
    invoices_sheet.append(invoice_data)

# Create the "Credits" sheet
credits_sheet = workbook.create_sheet('Credits')
credits_sheet.append(credit_headers)
for credit_data in credits:
    credits_sheet.append(credit_data)

# Create the "Kia Invoices" sheet
kia_sheet = workbook.create_sheet('Kia Invoices')
kia_sheet.append(invoice_headers)
for row in kia_invoices:
    kia_sheet.append(row)

# Save the workbook
workbook.save('/content/drive/MyDrive/Courage Kia/Invoice Rec/extracted_data.xlsx')

print('Data exported successfully.')


Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
/content/drive/MyDrive/Courage Kia/Invoice Rec/parts_invoices.csv does not exist. Skip is set to False.
Reading file: /content/drive/MyDrive/Courage Kia/Kia Invoices/Batch Report (40).pdf
Reading file: /content/drive/MyDrive/Courage Kia/Kia Invoices/Batch Report (41).pdf
Reading file: /content/drive/MyDrive/Courage Kia/Kia Invoices/Batch Report (42).pdf
Reading file: /content/drive/MyDrive/Courage Kia/Kia Invoices/Batch Report (43).pdf
Reading file: /content/drive/MyDrive/Courage Kia/Kia Invoices/Batch Report (44).pdf
Reading file: /content/drive/MyDrive/Courage Kia/Kia Invoices/Batch Report (45).pdf
Reading file: /content/drive/MyDrive/Courage Kia/Kia Invoices/Batch Report (46).pdf
Reading file: /content/drive/MyDrive/Courage Kia/Kia Invoices/Batch Report (47).pdf
Reading file: /content/drive/MyDrive/Courage Kia/Kia Invoices/Batch Report (48).pdf
Reading fil

In [ ]:
import openpyxl
from itertools import islice

# Open the workbook
workbook = openpyxl.load_workbook('/content/extracted_data.xlsx')

# Get the 'Invoices' sheet
invoices_sheet = workbook['Invoices']
credits_sheet = workbook['Credits']

# Create a dictionary to store the invoice totals
invoice_totals = {}
has_discrepancies = False  # initialize flag variable

# Iterate through the 'Invoices' sheet, adding up the price values for each invoice
for i, row in enumerate(invoices_sheet.iter_rows(min_row=1, values_only=True), start=1):
    if i == 1:
        continue  # skip header row
    invoice_no = row[0]
    price = row[10]
    if invoice_no not in invoice_totals:
        invoice_totals[invoice_no] = 0
    if price is not None:
        try:
            price = float(price)
        except ValueError:
            print(f"Could not convert price {price} to float in row {i}")
        else:
            invoice_totals[invoice_no] += price

for i, row in enumerate(credits_sheet.iter_rows(min_row=1, values_only=True), start=1):
    if i == 1:
        continue  # skip header row
    invoice_no = row[0]
    price = row[10]
    if invoice_no not in invoice_totals:
        invoice_totals[invoice_no] = 0
    if price is not None:
        try:
            price = float(price)
        except ValueError:
            print(f"Could not convert price {price} to float in row {i}")
        else:
            invoice_totals[invoice_no] += price

# Round the invoice totals to two decimal places
for invoice_no in invoice_totals:
    invoice_totals[invoice_no] = round(invoice_totals[invoice_no], 2)

# Get the 'Kia Invoices' sheet
kia_sheet = workbook['Kia Invoices']

# Iterate through the 'Kia Invoices' sheet, comparing the subtotal value to the invoice total for each invoice
for row in islice(kia_sheet.iter_rows(min_row=1, values_only=True), 1, None):
    invoice_no = row[0]
    subtotal = row[4]
    if invoice_no in invoice_totals:
        if subtotal != invoice_totals[invoice_no]:
            print(f"Invoice {invoice_no} subtotal does not match invoice total: {subtotal} vs {invoice_totals[invoice_no]}")
            has_discrepancies = True  # set flag variable to True
    elif invoice_no not in invoice_totals:
        print(f"No invoice total found for invoice {invoice_no}")
        has_discrepancies = True  # set flag variable to True

if not has_discrepancies:  # check flag variable
    print("All Invoices Reconcile")

FileNotFoundError: ignored



---
**Packing List Work in Process**





In [ ]:
import csv
import re
from pdfminer.high_level import extract_text

pdf_path = '/content/packing list.pdf'

# Extract text from PDF file
text = extract_text(pdf_path)

# Split text into lines and remove any empty lines
lines = [line.strip() for line in text.split('\n') if line.strip()]

# Split each line into columns using regex
columns = []
for line in lines:
    # Use regex to split line into columns
    # Here, we split the line based on tab characters and multiple spaces
    # You can adjust this regex as needed to split the line correctly
    cols = re.split(r'\t| {2,}', line)
    # Remove empty columns and append to list of columns
    columns.extend([col for col in cols if col.strip()])

# Write columns to CSV file
with open('/content/packing list.csv', 'w', newline='') as csv_file:
    writer = csv.writer(csv_file, delimiter='\t')
    writer.writerow(columns)


In [ ]:
import csv
import re
import datetime
from pdfminer.high_level import extract_text

pdf_path = '/content/packing list.pdf'

# Extract text from PDF file
text = extract_text(pdf_path)

# Split text into lines and remove any empty lines
lines = [line.strip() for line in text.split('\n') if line.strip()]

# Split each line into columns using regex
columns = []
for line in lines:
    # Use regex to split line into columns
    # Here, we split the line based on tab characters and multiple spaces
    # You can adjust this regex as needed to split the line correctly
    cols = re.split(r'\t| {2,}', line)
    # Remove empty columns and append to list of columns
    columns.extend([col for col in cols if col.strip()])

# Find the index of the column containing today's date
today = datetime.date.today().strftime('%m/%d/%Y')
date_col_index = None
for i, col in enumerate(columns):
    if col.endswith(today):
        date_col_index = i
        break

# If no column contains today's date, exit the script
if date_col_index is None:
    print(f"No data found for today's date: {today}")
    exit()

# Extract data from rows that end with today's date
rows = []
for line in lines:
    cols = re.split(r'\t| {2,}', line)
    # Only add the row if it ends with today's date
    if cols[date_col_index].endswith(today):
        rows.append(cols)

# Write rows to CSV file
with open('/content/packing list.csv', 'w', newline='') as csv_file:
    writer = csv.writer(csv_file, delimiter='\t')
    # Write header row
    writer.writerow(columns)
    # Write data rows
    for row in rows:
        writer.writerow(row)


IndexError: ignored